# Regressão - Previsão do valor de um ativo no dia seguinte

## Autores
- Felipe Bakowski Nantes de Souza  
- Vinicius Grecco Fonseca Mulato  
- Victor Soares


# 1.    Data set - Seleção

# 2. Data set - Explicação

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlp import mlp

# Lê o arquivo Excel
df = pd.read_excel('SP_MEMBERS.xlsx')

# Converte e salva em formato Parquet
df.to_parquet('SP_MEMBERS.parquet', index=False)

ArrowTypeError: ("Expected bytes, got a 'datetime.datetime' object", 'Conversion failed for column Data inicial with type object')